In [ ]:
print("Project 1")

In [ ]:
#Trading strategy with moving average 
#TODO: Changing strategy to trade every 5 years (on a Wednesday?); shorten training data
#Total number of trades with current code: 26
#PSR: 0.055 (can improve)

class TradingStrategy(QCAlgorithm):
    #Keep track of highest price for the stock 
    highestSPYPrice = 0
    def Initialize(self):
        self.SetCash(100000)  # Set Strategy Cash
        self.averages = {}
        self.spy = self.AddEquity("SPY",Resolution.Daily)
        self.spy.SetDataNormalizationMode(DataNormalizationMode.Raw)


    def OnData(self, data):
         if self.Securities["SPY"].Close > self.highestSPYPrice:
             self.highestSPYPrice = self.Securities["SPY"].Close
        #Check the moving average of the stock to decide if trading or holding 
         self.SelectionFunction(self.spy)
            
    def SelectionFunction(self, stock):
        #Earliest data available for the 3 stocks
        #Change accordingly for different timeslots (start_date starting at 1998,1,1)
        start_date = datetime(1998, 1, 1)
        #Up to the start of 2022 (need to shorten)
        end_date = datetime(2022, 1, 1)
        symbol = stock.Symbol
        #History for stock with daily resolution
        history = self.History(symbol, start_date, end_date, Resolution.Daily)
        #The tutorials did something with the code below but equity doesnt have AdjustedPrice
        #self.averages[symbol].update(self.Time, stock.AdjustedPrice)
        self.averages[symbol] = SelectionData(history)
        self.Debug(str(history))
        if not self.Time.weekday() == 1:
            return
        #Decide if trade or hold
        if  self.averages[symbol].is_ready() and self.averages[symbol].fast > self.averages[symbol].slow:
            #Set asset holdings to 100% of symbol
            self.SetHoldings(symbol, 1)
        else: 
            #Liquidate assets
            self.Liquidate(symbol)
            #self.Liquidate(tag = "Liquidated")

#Moving average models
class SelectionData():
    def __init__(self, history):
        #Not sure if it should be 100 or 200 (I had both numbers in my notes)
        self.slow = ExponentialMovingAverage(200)
        self.fast = ExponentialMovingAverage(50)
        #4. Loop over the history data and update the indicators
        for bar in history.itertuples():
            self.fast.Update(bar.Index[1], bar.close)
            self.slow.Update(bar.Index[1], bar.close)

    def is_ready(self):
        return self.slow.IsReady and self.fast.IsReady

In [ ]:
# Trading strategy with Momentum Indicators

from AlgorithmImports import *

class MomentumBasedTacticalAllocation(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2013, 1, 1)
        self.SetEndDate(2018, 1, 1)
        self.SetCash(100000)

        self.apple = self.AddEquity("KO", Resolution.Daily)
        self.bnd = self.AddEquity("BND", Resolution.Daily)
        self.applemomentum = self.MOMP("KO", 50, Resolution.Daily)
        self.bondMomentum = self.MOMP("BND", 50, Resolution.Daily)
        self.SetBenchmark("KO")
        self.SetWarmUp(50)

    def OnData(self, data):

        # Don't place trades until our indicators are warmed up:
        if self.IsWarmingUp:
            return

        #1. If SPY has more upward momentum than BND, then we liquidate our holdings in BND and allocate 100% of our equity to SPY
        if self.applemomentum.Current.Value > self.bondMomentum.Current.Value:
            self.Liquidate("BND")
            self.SetHoldings("KO", 1)
        else:
            self.Liquidate("KO")
            self.SetHoldings("BND", 1)